# Azure AI Search quickstart (AAD with DefaultAzureCredential)
This notebook shows how to use AAD (no keys) to query an Azure AI Search index securely, with robust error handling. It follows a reproducible and testable structure.

In [ ]:
# 1) Install and Import Dependencies
!pip install -q python-dotenv azure-identity azure-search-documents ipytest

In [ ]:
# Imports
import os, random, json, time, logging, pathlib
from datetime import datetime
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
load_dotenv()


In [ ]:
# 2) Configure Environment and Reproducibility
random.seed(42)
logging.basicConfig(level=logging.INFO)
SEARCH_SERVICE = os.getenv('AZURE_SEARCH_SERVICE')
SEARCH_INDEX = os.getenv('AZURE_SEARCH_INDEX')
assert SEARCH_SERVICE, "Set AZURE_SEARCH_SERVICE in .env"
assert SEARCH_INDEX, "Set AZURE_SEARCH_INDEX in .env"

In [ ]:
# 3) Load or Generate Data (optional inspection)
from pathlib import Path
DATA = Path('..') / 'data' / 'payments' / 'sample_transactions.csv'
print('Sample data exists:', DATA.exists(), DATA)

In [ ]:
# 4) Implement Core Functions or Classes
from typing import List, Dict, Any
from azure.search.documents.models import QueryType
def get_search_client() -> SearchClient:
    endpoint = f"https://{SEARCH_SERVICE}.search.windows.net"
    cred = DefaultAzureCredential()
    return SearchClient(endpoint=endpoint, index_name=SEARCH_INDEX, credential=cred)

def query_search(q: str, top: int = 5) -> List[Dict[str, Any]]:
    client = get_search_client()
    results = client.search(search_text=q, query_type=QueryType.SIMPLE, top=top)
    return [dict(r) for r in results]

In [ ]:
# 5) Unit Tests (ipytest)
import ipytest
ipytest.autoconfig()

%%ipytest -qq
def test_query_handles_results():
    import types
    # Smoke test: function should return a list (may be empty in local env)
    try:
        res = query_search("payment")
        assert isinstance(res, list)
    except Exception as ex:
        # In case env is not configured, ensure meaningful error
        assert any(s in str(ex).lower() for s in ["unauthorized", "forbidden", "azure", "credential", "index"])

In [ ]:
# 6) Run Core Workflow
try:
    results = query_search("chargeback", top=3)
    print(f"Returned {len(results)} docs")
    for r in results:
        print(list(r.keys())[:5])
except Exception as ex:
    print("Query failed:", ex)

In [ ]:
# 7) Visualize Results
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.DataFrame(results) if 'results' in globals() else pd.DataFrame([])
display(df.head())
if not df.empty:
    df['dummy_metric'] = 1
    df['dummy_metric'].value_counts().plot(kind='bar', title='Result Count')
    plt.show()

In [ ]:
# 8) Save Outputs and Artifacts
from datetime import datetime
ts = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
out = pathlib.Path('outputs'); out.mkdir(exist_ok=True, parents=True)
p = out / f"results_{ts}.json"
if 'results' in globals():
    p.write_text(json.dumps(results, indent=2))
    print('Saved', p)
else:
    print('No results to save')

In [ ]:
# 9) CLI/Script Entrypoint (optional)
if __name__ == "__main__":
    try:
        print(query_search("refund", top=2))
    except Exception as ex:
        print("Error:", ex)

In [ ]:
# 10) Clean Up and Resource Release
# Nothing to clean up for Search client in this example.
pass